### Word Embeddings

We need to vectorize the words to use in machine learning models. The simplest approach is to assign a number to each word but this implies meaningless relations between words such as "happy > excited > notebook" if happy is assigned a higher number than other words. Alternative approach is to use one-hot encoding where each word is represented as a dimension in a $|V|$-dimensional space. Though this representation does not imply meaningless semantic relations, it does not reflect any meaning as well. Since each word has a separate dimension, each word is equidistant with each other. Besides, one-hot encoded vectors do not scale well with respect to $|V|$ and create huge vectors in real-life applications ($|V| \approx 1M$.

**Word embeddings** have usually much lower dimensions ($ \approx 100 - 1000$) than the previous representations and carry semantic knowledge. For instance, we can represent words in semantic dimensions such as positivity and concreteness of words. In such space, puppy and kitten are much closer to each other than puppy anger. With word embeddings, we can represent the meaning of words/texts more accurately and scale to real-life problems.

<img src='images/2d_space.png' style="zoom: 30%" />

**Remark:** In reality, we do not know what each dimension means. The word embedding model learns each dimension by itself. 

Word embeddings are learned through a large corpus with no labels. They assume that words occur in similar contexts have similar meaning and should be represented with similar vectors. Thus, they deduce the labels from the context in the unlabeled corpus. This is called *self-supervised* learning, since there is no explicit labels as in supervised learning but the labels are derived from the data itself.

**Note:** Depending on the application, word embeddings can be learned through a domain-specific corpus such as contracts, movie scripts or a general corpus such as Wikipedia.

Popular word embeddings models include Word2Vec (continuous bag-of-words, skip-gram), GloVe, and fastText. These techniques create a single embedding for each word and pre-trained vectors are available online. More recent models, such as BERT, ELMo, and GPT, learn contextual representations for words and embed the words with its context. Pre-trained models are available and can be fine-tuned with the task at hand.

### CBOW Model

In continuous bag-of-words (CBOW) we train a shallow neural network to predict a word given its corpus. The rationale is words occur in similar contexts are also similar. For instance, "The little \_\_\_\__ is barking" can be filled with dog, puppy, hound and thus, these words should have similar vectors. 

We create training samples to feed the neural network based on this approach. We first set a context size to decide which words to include in the context and then deduce input-output pairs from the sentences such that the input is the context words and the outpus is the central word. From a sentence "I am happy because I am learning" we deduce the pairs {(I am because I, happy),(am happy I am, because), (happy because am learning, I)} for training, with a half-context size of 2 (or equivalently, window size of 5). Last, we vectorize these pairs by one-hot encoding and average the context word vectors to obtain a single input vector.

CBOW uses a shallow neural network to learn word embeddings using the previous formulation. It has an input layer of size $V \times m$, where $m$ is the training batch size. It maps these inputs to the hidden layer by computing $H = relu(W_1 X + B_1)$, which is an $N \times m$ matrix. Last, it classifies the hidden states by calculating $\hat{Y} = W_2 H + B_2$, which is a $V \times m$ matrix. Here, first column of $\hat{Y}$ corresponds to the prediction of the first column of $X$

**Remark:** $X$ is constructed by stacking the input vectors column by column. In other words, in $X$ each column is a sample, unlike most models. If the samples were placed in rows, then the matrix multiplication would use transposed weights.

**Remark:** ReLU and softmax are hyper-parameters and can be changed with other alternatives as well. *Note:* $relu(x) = max(x, 0)$ and $softmax(z_i) = \frac{e^{z_i}}{\sum_{j=1}^V e^{z_j}$. The output of softmax sums up to 1 and each element can be intepreted as the probability of each word being the correct label

<img src='images/cbow.png' style='zoom: 40%' />

We need a loss function to update the weights and biases. We use cross-entropy loss, which is a quite common loss function for multi-class classfication problems. The cross-entropy loss of a single sample is defined as:

$$
J = - \sum_{k=1}^{V} y_k \log{\hat{y_k}}
$$

Note that the loss is non-zero only for the actual label since $y_k=0$ otherwise. Therefore, cross-entropy boils down to sum of negative log predictions corresponding to actual labels. The negative log prediction is close to $0$ when the prediction is close to $1$ and close to $\infty$ otherwise. Therefore, minimizing the cross-entropy encourages predictions to be close to actual label, as desired.

We can compute the loss of a batch by averaging the $J$ of each sample and minimize the overall loss using backpropagation and gradient descent. $$ J_{batch} = -\frac{1}{m} \sum_{i=1}^{m} \sum_{j=1}^{V} y_j^{(i)} \log{\hat{y}_j^{(i)}} = -\frac{1}{m} \sum_{i=1}^{m} J^{(i)} 
$$

In the end of training, $W_1$ and $W_2$ carries embeddings for each word in the columns and rows, respectibely. We can either use them separately or average them to obtain a single word embedding. In this course, we choose the second option and compute the embedding matrix  $E = \frac{1}{2} (W_1 + W_2^T)$. Here, $i^{th}$ column of $E$ corresponds to the embedding of the $i^{th}$ word in the vocabulary.

### Evaluation

We can evaluate the learned word embeddings in two different ways: *intrinsic* and *extrinsic* evaluation. In the intrinsic evalation, we judge semantic capabilities of the embeddings by using them on analogy task or analyzing the clusters in the space. This is an easy and direct way of evaluation, yet it requires a labeled data. In the extrinsic evaluation, we use the learned embeddings in the task we are interested in and measure the task performance. This is much more time consuming since it requires training additional model for the task and harder to troubleshoot, since it does not inform about the error sources.